# ETL Project
## Terrence Cummings
Scraping race mix by Minneapolis neighborhood

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
import time
import sys
import os
import datetime
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")

In [156]:
#Read in CSV's from Extract
cmnty_links_df=pd.read_csv('cmnty_links.csv')
nbhd_links_df=pd.read_csv('nbhd_links.csv')
cmnty_race_df=pd.read_csv('cmnty_race.csv')
nbhd_race_df=pd.read_csv('nbhd_race.csv')
cmnty_keys_df=pd.read_csv('cmnty_keys.csv')
nbhd_keys_df=pd.read_csv('nbhd_keys.csv')

In [157]:
nbhd_race_df = nbhd_race_df.replace('suppressed', np.nan)


In [158]:
nbhd_race_df['white_cnt'] = nbhd_race_df['white_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['black_cnt'] = nbhd_race_df['black_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['native_cnt'] = nbhd_race_df['native_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['asian_cnt'] = nbhd_race_df['asian_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['other_cnt'] = nbhd_race_df['other_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['two_or_more_cnt'] = nbhd_race_df['two_or_more_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['hispanic_cnt'] = nbhd_race_df['hispanic_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['of_color_cnt'] = nbhd_race_df['of_color_cnt'].str.replace(',', '').astype(float)

In [159]:
nbhd_race_df['white_pct'] = nbhd_race_df['white_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['black_pct'] = nbhd_race_df['black_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['native_pct'] = nbhd_race_df['native_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['asian_pct'] = nbhd_race_df['asian_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['other_pct'] = nbhd_race_df['other_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['two_or_more_pct'] = nbhd_race_df['two_or_more_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['hispanic_pct'] = nbhd_race_df['hispanic_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['of_color_pct'] = nbhd_race_df['of_color_pct'].str.replace('%', '').astype(float)/100

In [160]:
nbhd_race_df['total_cnt']=round(nbhd_race_df['white_cnt']/nbhd_race_df['white_pct'],0)

In [161]:
temp_nbhd_race_df = nbhd_race_df.merge(nbhd_keys_df, how='outer', left_on='neighborhood', right_on='neighborhood_url')
final_nbhd_race_df=temp_nbhd_race_df.drop(['neighborhood', 'neighborhood_url'], axis = 1) 
final_nbhd_race_df.set_index('NBHD_KEY', inplace=True)


In [162]:
final_nbhd_race_df = final_nbhd_race_df[['NEIGHBORHOOD','COMMUNITY','total_cnt','white_cnt','white_pct','black_cnt','black_pct','native_cnt','native_pct','asian_cnt','asian_pct','other_cnt','other_pct','two_or_more_cnt','two_or_more_pct','hispanic_cnt','hispanic_pct','of_color_cnt','of_color_pct','URL']]


In [163]:
final_nbhd_race_df

,NEIGHBORHOOD,COMMUNITY,total_cnt,white_cnt,white_pct,black_cnt,black_pct,native_cnt,native_pct,asian_cnt,asian_pct,other_cnt,other_pct,two_or_more_cnt,two_or_more_pct,hispanic_cnt,hispanic_pct,of_color_cnt,of_color_pct,URL
NBHD_KEY,,,,,,,,,,,,,,,,,,,,
N71,ARMATAGE,SOUTHWEST,5108.0,4316.0,0.845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185.0,0.036,NaN,NaN,792.0,0.155,http://www.mncompass.org/profiles/neighborhood...
N46,AUDUBON PARK,NORTHEAST,5119.0,4034.0,0.788,386.0,0.075,NaN,NaN,NaN,NaN,NaN,NaN,303.0,0.059,449.0,0.088,1083.0,0.212,http://www.mncompass.org/profiles/neighborhood...
N63,BANCROFT,POWDERHORN,3544.0,2601.0,0.734,413.0,0.116,NaN,NaN,74.0,0.021,NaN,NaN,278.0,0.078,521.0,0.147,943.0,0.266,http://www.mncompass.org/profiles/neighborhood...
N02,CEDAR-ISLES-DEAN,CALHOUN-ISLE,3195.0,2927.0,0.916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.mncompass.org/profiles/neighborhood...
N80,CEDAR RSIDE/WEST BANK,UNIVERSITY,10059.0,3219.0,0.320,5146.0,0.511,NaN,NaN,1156.0,0.115,NaN,NaN,396.0,0.039,270.0,0.027,6846.0,0.680,http://www.mncompass.org/profiles/neighborhood...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N55,ST ANTHONY EAST,NORTHEAST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N56,ST ANTHONY WEST,NORTHEAST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
N33,SUMNER-GLENWOOD,NEAR NORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [164]:
cmnty_race_df['community'] = cmnty_race_df['community'].str.upper()
cmnty_race_df = cmnty_race_df.replace('suppressed', '0')
cmnty_race_df = cmnty_race_df.replace(np.nan, '0', regex=True)


In [165]:
cmnty_race_df['white_cnt'] = cmnty_race_df['white_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['black_cnt'] = cmnty_race_df['black_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['native_cnt'] = cmnty_race_df['native_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['asian_cnt'] = cmnty_race_df['asian_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['other_cnt'] = cmnty_race_df['other_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['two_or_more_cnt'] = cmnty_race_df['two_or_more_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['hispanic_cnt'] = cmnty_race_df['hispanic_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['of_color_cnt'] = cmnty_race_df['of_color_cnt'].str.replace(',', '').astype(float)


In [166]:
cmnty_race_df['white_pct'] = cmnty_race_df['white_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['black_pct'] = cmnty_race_df['black_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['native_pct'] = cmnty_race_df['native_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['asian_pct'] = cmnty_race_df['asian_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['other_pct'] = cmnty_race_df['other_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['two_or_more_pct'] = cmnty_race_df['two_or_more_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['hispanic_pct'] = cmnty_race_df['hispanic_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['of_color_pct'] = cmnty_race_df['of_color_pct'].str.replace('%', '').astype(float)/100

In [167]:
cmnty_dict = {'CALHOUN-ISLES': 'CALHOUN-ISLE', 'CAMDEN':'CAMDEN', 'CENTRAL': 'CENTRAL', 'LONGFELLOW': 'LONGFELLOW', 'NEAR-NORTH': 'NEAR NORTH', 'NOKOMIS':'NOKOMIS', 'NORTHEAST':'NORTHEAST', 'PHILLIPS': 'PHILLIPS', 'POWDERHORN': 'POWDERHORN', 'SOUTHWEST':'SOUTHWEST', 'UNIVERSITY':'UNIVERSITY'}

In [168]:
cmnty_race_df = cmnty_race_df.replace({"community": cmnty_dict})

In [169]:
final_cmnty_race_df = cmnty_race_df.merge(cmnty_keys_df, how='left', left_on='community', right_on='COMMUNITY')
del final_cmnty_race_df['COMMUNITY']
final_cmnty_race_df.set_index('CMNTY_KEY', inplace=True)
final_cmnty_race_df['total_cnt']=round(final_cmnty_race_df['white_cnt']/final_cmnty_race_df['white_pct'],0)


In [170]:
final_cmnty_race_df=final_cmnty_race_df.drop(['white_pct','black_pct','native_pct','asian_pct','other_pct','two_or_more_pct','hispanic_pct','of_color_pct'], axis = 1) 

final_cmnty_race_df = final_cmnty_race_df[['community','total_cnt','white_cnt','black_cnt','native_cnt','asian_cnt','other_cnt','two_or_more_cnt','hispanic_cnt','of_color_cnt', 'URL']]

In [171]:
final_cmnty_race_df

,community,total_cnt,white_cnt,black_cnt,native_cnt,asian_cnt,other_cnt,two_or_more_cnt,hispanic_cnt,of_color_cnt,URL
CMNTY_KEY,,,,,,,,,,,
C00,CALHOUN-ISLE,32417.0,27911.0,1591.0,0.0,1080.0,416.0,1309.0,1166.0,4512.0,http://www.mncompass.org/profiles/communities/...
C01,CAMDEN,30760.0,13565.0,10677.0,343.0,3500.0,691.0,1953.0,2360.0,17164.0,http://www.mncompass.org/profiles/communities/...
C02,CENTRAL,35323.0,23384.0,7133.0,405.0,2410.0,410.0,1564.0,1645.0,11928.0,http://www.mncompass.org/profiles/communities/...
C03,LONGFELLOW,29017.0,20399.0,5521.0,290.0,815.0,901.0,1072.0,2083.0,8613.0,http://www.mncompass.org/profiles/communities/...
C04,NEAR NORTH,35930.0,8264.0,17163.0,629.0,5493.0,1864.0,2448.0,3784.0,27604.0,http://www.mncompass.org/profiles/communities/...
C05,NOKOMIS,38800.0,30885.0,3507.0,629.0,776.0,1214.0,1779.0,2160.0,7905.0,http://www.mncompass.org/profiles/communities/...
C06,NORTHEAST,37451.0,28051.0,3921.0,674.0,1035.0,1999.0,1793.0,3989.0,9422.0,http://www.mncompass.org/profiles/communities/...
C07,PHILLIPS,22063.0,6310.0,9606.0,1255.0,757.0,3400.0,750.0,5709.0,15768.0,http://www.mncompass.org/profiles/communities/...
C08,POWDERHORN,57183.0,33738.0,11305.0,787.0,1387.0,7056.0,2876.0,13036.0,23438.0,http://www.mncompass.org/profiles/communities/...
